In [4]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
import spacy

spacy.require_gpu()
nlp = spacy.load("en_core_web_lg")


In [14]:
import collect_data
import json
config = json.load(open('twitter_config.json'))['cache_index']
config.pop('snapshot_path')
config.pop('batch_size')


cursor,doc_count = collect_data.scroll_index(**config)

In [15]:
res = []
for i,s in enumerate(cursor):
    if i == 100: break 
    res.append(s)

In [5]:
SNAPSHOTS_BASE = Path('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/snapshots/')
# dd = pd.read_json()
for f in sorted(SNAPSHOTS_BASE.glob('*.pkl')): break

dd = pd.read_pickle(f)

In [12]:

import pickle

toks = pickle.load(open('tokens/00100000.pkl','rb'))

In [7]:
import time
start = time.perf_counter()
get_tokens(dd.whole_text)
print(f'whole text took {time.perf_counter() - start}')




whole text took 97.2067287825048


In [10]:
start = time.perf_counter()

for i in tqdm(range(0,len(dd),100) ):
    get_tokens(dd.whole_text[i:i+100])
    # print(len(dd['whole_text']))
print(f'batched text took {time.perf_counter() - start}')


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
from collect_data import load_cache

raw_data = load_cache('/data/blockchain-interoperability/blockchain-social-media/analysis/twitter/snapshots/')



100%|██████████| 43/43 [02:49<00:00,  3.94s/it]


In [9]:
raw_data.id.unique().shape,raw_data.shape

((4262198,), (4262198, 16))

In [10]:
raw_data.head()

,in_reply_to_status_id,created_at,truncated,timestamp_ms,reply_count,retweet_count,entities.hashtags,favorite_count,id,text,lang,quote_count,user.description,user.id,extended_tweet.entities.hashtags,extended_tweet.full_text
0,1.591014e+18,2022-11-11 12:03:53+00:00,False,2022-11-11 12:03:53.965,0,0,[],0,1591039006850310148,"@0x5ab0 @Uniswap oh, missed fiat part, just sa...",en,0,@basinDAO cures my obsessions with #naturalcap...,333239871,None,None
1,NaN,2022-11-11 12:03:53+00:00,True,2022-11-11 12:03:53.946,0,0,[],0,1591039006770622468,Never seen such a deal for a punk since 2017! ...,en,0,"Зөөгчөө нэг шоот хайр авъя,.. #NoWar #SlavaUkr...",1590345908172083201,[],Never seen such a deal for a punk since 2017! ...
2,NaN,2022-11-11 12:03:53+00:00,True,2022-11-11 12:03:53.882,0,0,[],0,1591039006501994503,Collection: ens \n Sold for: 1.0040189 Eth (12...,en,0,Tracks sale/purchase of whale NFT accounts \...,1520781667769614337,"[{'indices': [169, 173], 'text': 'ens'}, {'ind...",Collection: ens \n Sold for: 1.0040189 Eth (12...
3,NaN,2022-11-10 06:05:54+00:00,False,NaT,0,4,"[{'indices': [56, 69], 'text': 'LoveOnTourLA'}...",5,1590586526303686657,🎥 Harry singing Music For A Sushi Restaurant !...,en,0,"Accept what is, let go of what was, and have f...",786084425406832644,None,None
4,NaN,2022-11-11 12:03:54+00:00,True,2022-11-11 12:03:54.075,0,0,[],0,1591039007311683585,AAVE Airdrop iis now live. You can easily clai...,en,0,Kunji is meant to help people who want to pass...,1589700910883676160,[],AAVE Airdrop iis now live. You can easily clai...


If trucated is False, that means text has the full text. If True, extended_tweet.full_text has the full text.


Note: scroll query to get everything in index

In [12]:
raw_data.columns

Index(['in_reply_to_status_id', 'created_at', 'truncated', 'timestamp_ms',
       'reply_count', 'retweet_count', 'entities.hashtags', 'favorite_count',
       'id', 'text', 'lang', 'quote_count', 'user.description', 'user.id',
       'extended_tweet.entities.hashtags', 'extended_tweet.full_text'],
      dtype='object')

In [15]:
(raw_data['whole_text'] == '').any()

# very good!

False